In [1]:
from typing import List, Optional
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import scipy.stats as st
from scipy.stats import probplot, ks_2samp, shapiro, ttest_ind, mannwhitneyu, chi2_contingency, ttest_rel
from statsmodels.stats.weightstats import zconfint

# импортирование библиотек моделей
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb, lightgbm as lgbm, catboost as catb

from sklearn.metrics import roc_auc_score, r2_score, classification_report
from sklearn.model_selection import KFold, cross_val_score, train_test_split, ShuffleSplit
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, learning_curve, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
import missingno as msno
%matplotlib inline

# убираем лишние замечания интерпретатора
import warnings
warnings.simplefilter('ignore')

# import additional packages for WOE/IV
import pandas.core.algorithms as algos
import scipy.stats.stats as stats
import re
import traceback
import string
# глобальные переменные WOE/IV
max_bin = 20
force_bin = 3

In [2]:
# функции метода WOE/IV

def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [3]:
def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred))
    print('TEST\n\n' + classification_report(y_test_true, y_test_pred))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [4]:
# dataset loading
train = pd.read_csv("/home/pi/GeekBrains/Kaggle/competition/train.csv")
test = pd.read_csv("/home/pi/GeekBrains/Kaggle/competition/test.csv")
client_profile = pd.read_csv("/home/pi/GeekBrains/Kaggle/competition/client_profile.csv")
bki = pd.read_csv("/home/pi/GeekBrains/Kaggle/competition/bki.csv")
payments = pd.read_csv("/home/pi/GeekBrains/Kaggle/competition/payments.csv")
application_history = pd.read_csv("/home/pi/GeekBrains/Kaggle/competition/applications_history.csv")

print("train.shape = {} rows, {} cols".format(*train.shape))
print("test.shape = {} rows, {} cols".format(*test.shape))
print("client_profile.shape = {} rows, {} cols".format(*client_profile.shape))
print("bki.shape = {} rows, {} cols".format(*bki.shape))
print("payments.shape = {} rows, {} cols".format(*payments.shape))
print("application_history.shape = {} rows, {} cols".format(*application_history.shape))

train.shape = 110093 rows, 3 cols
test.shape = 165141 rows, 2 cols
client_profile.shape = 250000 rows, 24 cols
bki.shape = 945234 rows, 17 cols
payments.shape = 1023932 rows, 8 cols
application_history.shape = 1670214 rows, 26 cols


In [5]:
X_train = train.copy()
X_test = test.copy()
X_client_profile = client_profile.copy()
X_bki = bki.copy()
X_payments = payments.copy()
X_application_history = application_history.copy()

X_train = X_train.sort_values(by = 'APPLICATION_NUMBER')
X_train = X_train.reset_index(drop=True)
X_bki = X_bki.sort_values(by = 'APPLICATION_NUMBER')
X_bki = X_bki.reset_index(drop=True)
X_client_profile = X_client_profile.sort_values(by = 'APPLICATION_NUMBER')
X_client_profile = X_client_profile.reset_index(drop=True)
X_payments = X_payments.sort_values(by = 'APPLICATION_NUMBER')
X_payments = X_payments.reset_index(drop=True)
X_application_history = X_application_history.sort_values(by = 'APPLICATION_NUMBER')
X_application_history = X_application_history.reset_index(drop=True)
X_test = X_test.sort_values(by = 'APPLICATION_NUMBER')
X_test = X_test.reset_index(drop=True)

print("APPLICATION_NUMBER_train = {} ".format(X_train['APPLICATION_NUMBER'].nunique()))
print("APPLICATION_NUMBER_test = {} ".format(X_test['APPLICATION_NUMBER'].nunique()))
print("APPLICATION_NUMBER_client_profile = {} ".format(X_client_profile['APPLICATION_NUMBER'].nunique()))
print("APPLICATION_NUMBER_bki = {} ".format(X_bki['APPLICATION_NUMBER'].nunique()))
print("APPLICATION_NUMBER_payments = {} ".format(X_payments['APPLICATION_NUMBER'].nunique()))
print("APPLICATION_NUMBER_application_history = {} ".format(X_application_history['APPLICATION_NUMBER'].nunique()))

APPLICATION_NUMBER_train = 110093 
APPLICATION_NUMBER_test = 165141 
APPLICATION_NUMBER_client_profile = 250000 
APPLICATION_NUMBER_bki = 273131 
APPLICATION_NUMBER_payments = 264726 
APPLICATION_NUMBER_application_history = 338857 


In [6]:
X_train_p = pd.merge(X_train, X_client_profile, how='outer')
X_train_p = X_train_p.sort_values(by = ['APPLICATION_NUMBER'])
X_train_p = X_train_p.reset_index(drop=True)
X_test_p = pd.merge(X_test, X_client_profile, how='outer')
X_test_p = X_test_p.sort_values(by = ['APPLICATION_NUMBER'])
X_test_p = X_test_p.reset_index(drop=True)
X_train_p.shape

(270554, 26)

In [7]:
X_train_p.loc[X_train_p['GENDER'].isna(), 'GENDER'] = 'nan'
X_test_p.loc[X_test_p['GENDER'].isna(), 'GENDER'] = 'nan'
X_train_p.loc[X_train_p['EDUCATION_LEVEL'].isna(), 'EDUCATION_LEVEL'] = 'nan'
X_test_p.loc[X_test_p['EDUCATION_LEVEL'].isna(), 'EDUCATION_LEVEL'] = 'nan'
X_train_p.loc[X_train_p['FAMILY_STATUS'].isna(), 'FAMILY_STATUS'] = 'nan'
X_test_p.loc[X_test_p['FAMILY_STATUS'].isna(), 'FAMILY_STATUS'] = 'nan'
X_test_p.shape

(280965, 25)

X_train_p['NAME_CONTRACT_TYPE'] = X_train_p['NAME_CONTRACT_TYPE'].map({'Cash':'71', 'Credit Card':'72'}).astype('int16')
X_test_p['NAME_CONTRACT_TYPE'] = X_test_p['NAME_CONTRACT_TYPE'].map({'Cash':'71', 'Credit Card':'72'}).astype('int16')
X_train_p['GENDER'] = X_train_p['GENDER'].map({'nan':'40', 'M':'41',
                                                    'F':'42', 'XNA':'43'}).astype('float32')
X_train_p['EDUCATION_LEVEL'] = X_train_p['EDUCATION_LEVEL'].map({'nan':'50', 
       'Secondary / secondary special':'51', 'Higher education':'52', 'Incomplete higher':'53',
        'Lower secondary':'54', 'Academic degree':'55'}).astype('float32')
X_train_p['FAMILY_STATUS'] = X_train_p['FAMILY_STATUS'].map({'nan':'60', 'Single / not married':'61',
       'Married':'62', 'Widow':'63', 'Civil marriage':'64',
       'Separated':'65', 'Unknown':'66'}).astype('float32')
X_test_p['GENDER'] = X_train_p['GENDER'].map({'nan':'40', 'M':'41', 'F':'42', 'XNA':'43'}).astype('float32')
X_test_p['EDUCATION_LEVEL'] = X_test_p['EDUCATION_LEVEL'].map({'nan':'50', 
       'Secondary / secondary special':'51', 'Higher education':'52', 'Incomplete higher':'53',
        'Lower secondary':'54', 'Academic degree':'55'}).astype('float32')
X_test_p['FAMILY_STATUS'] = X_test_p['FAMILY_STATUS'].map({'nan':'60', 'Single / not married':'61',
       'Married':'62', 'Widow':'63', 'Civil marriage':'64',
       'Separated':'65', 'Unknown':'66'}).astype('float32')

Flag_Train = X_train_p.nunique(axis=1)
X_train_p['FLAG'] = Flag_Train
Flag_Test = X_test_p.nunique(axis=1)
X_test_p['FLAG'] = Flag_Test

In [8]:
X_train_p = X_train_p.loc[~X_train_p['NAME_CONTRACT_TYPE'].isna()]
X_test_p = X_test_p.loc[~X_test_p['NAME_CONTRACT_TYPE'].isna()]

y_train = X_train_p['TARGET']
X_train_p = X_train_p.drop(['TARGET'], axis=1)
X_test_p.shape

(165141, 25)

In [9]:
X_trainn, X_testt, y_trainn, y_testt = train_test_split(X_train_p, y_train, shuffle=True, test_size=0.2, random_state=21)

In [10]:
model_catb = catb.CatBoostClassifier(cat_features=['NAME_CONTRACT_TYPE',
                                                  'GENDER', 'EDUCATION_LEVEL',
                                                  'FAMILY_STATUS'], silent=True, random_state=21)
model_catb.fit(X_trainn, y_trainn)

y_train_pred = model_catb.predict(X_trainn)
y_test_pred = model_catb.predict(X_testt)

get_classification_report(y_trainn, y_train_pred, y_testt, y_test_pred)

TRAIN

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96     80947
         1.0       0.94      0.07      0.14      7127

    accuracy                           0.92     88074
   macro avg       0.93      0.54      0.55     88074
weighted avg       0.93      0.92      0.89     88074

TEST

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96     20249
         1.0       0.45      0.02      0.03      1770

    accuracy                           0.92     22019
   macro avg       0.69      0.51      0.49     22019
weighted avg       0.88      0.92      0.88     22019

CONFUSION MATRIX

col_0     0.0  1.0
TARGET            
0.0     20214   35
1.0      1741   29


In [11]:
print(roc_auc_score(y_trainn, model_catb.predict_proba(X_trainn)[:,1]))
print(roc_auc_score(y_testt, model_catb.predict_proba(X_testt)[:,1]))

0.8288258105625964
0.7114397223494053


In [12]:
TARGET = pd.DataFrame(model_catb.predict_proba(X_test_p))
TARGET =TARGET.rename(columns={0:0, 1:'TARGET'}).drop(0, axis=1)
TARGET.head()

TARGET
0  0.028706
1  0.043946
2  0.071938
3  0.036477
4  0.005243

In [13]:
X_test['TARGET'] = TARGET

In [ ]:
X_test = X_test.drop(['NAME_CONTRACT_TYPE'], axis=1)
X_test.shape

In [ ]:
X_test.to_csv('test_submission.csv', index=False)

model_catb = catb.CatBoostClassifier(cat_features=['NAME_CONTRACT_TYPE', 'GENDER', 'EDUCATION_LEVEL', 'FAMILY_STATUS'], class_weights=[1, 10], random_state=21)

params = {'n_estimators':[50, 500, 1000, 1500],
          'max_depth':[3, 5, 7]}

cv=StratifiedKFold(n_splits=5, random_state=21, shuffle=True)

rs = RandomizedSearchCV(model_catb, params, scoring='f1', cv=cv, n_jobs=-1)
rs.fit(X_trainn, y_trainn)

rs.best_params_

rs.best_score_

final_model = catb.CatBoostClassifier(n_estimators=1500, max_depth=5,
                                      cat_features=['NAME_CONTRACT_TYPE', 'GENDER', 'EDUCATION_LEVEL', 'FAMILY_STATUS'], class_weights=[1, 10],
                                      silent=True, random_state=21)
final_model.fit(X_trainn, y_trainn)

y_train_pred = final_model.predict(X_trainn)
y_test_pred = final_model.predict(X_testt)

get_classification_report(y_trainn, y_train_pred, y_testt, y_test_pred)

In [14]:
X_bki.columns

Index(['APPLICATION_NUMBER', 'BUREAU_ID', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
       'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY'],
      dtype='object')

In [15]:
X_bki['CREDIT_TYPE'].unique()

array(['Consumer credit', 'Credit card', 'Car loan', 'Microloan',
       'Mortgage', 'Loan for working capital replenishment',
       'Another type of loan', 'Unknown type of loan',
       'Loan for business development', 'Cash loan (non-earmarked)',
       'Real estate loan', 'Loan for the purchase of equipment',
       'Loan for purchase of shares (margin lending)',
       'Mobile operator loan'], dtype=object)

X_bki_ = X_bki.loc[(X_bki['CREDIT_TYPE'] == 'Credit card')|(X_bki['CREDIT_TYPE'] == 'Cash loan (non-earmarked)'), :]
X_bki_.head()

In [17]:
X_train_pb = pd.merge(X_train_p, X_bki, how='outer')
X_train_pb = X_train_pb.sort_values(by = ['APPLICATION_NUMBER'])
X_train_pb = X_train_pb.loc[~X_train_pb['NAME_CONTRACT_TYPE'].isna()]
X_train_pb = X_train_pb.reset_index(drop=True)
X_train_pb.shape

(317272, 41)

In [18]:
X_test_pb = pd.merge(X_test_p, X_bki, how='outer')
X_test_pb = X_test_pb.sort_values(by = ['APPLICATION_NUMBER'])
X_test_pb = X_test_pb.loc[~X_test_pb['NAME_CONTRACT_TYPE'].isna()]
X_test_pb = X_test_pb.reset_index(drop=True)
X_test_pb.shape

(476367, 41)

In [ ]:
X_test_pb_ = X_test_pb.drop_duplicates(subset=['APPLICATION_NUMBER'], keep='first')
X_test_pb_.shape

In [ ]:
X_train_pb_ = X_train_pb.drop_duplicates(subset=['APPLICATION_NUMBER'], keep='first')
X_train_pb_.shape

In [ ]:
categorical_features = X_train_pb_.select_dtypes(include=['object'])
print(f"count of categorical_features {categorical_features.shape[1]}")

cat_features = categorical_features.columns.to_list()
cat_features

In [ ]:
X_train_pb_.loc[X_train_pb_['CREDIT_ACTIVE'].isna(), 'CREDIT_ACTIVE'] = 'nan'
X_train_pb_['CREDIT_ACTIVE'].unique()

X_train_pb_['CREDIT_ACTIVE'] = X_train_pb_['CREDIT_ACTIVE'].map({'nan':'0', 'Closed':'1',
                                                                 'Active':'2', 'Sold':'3',
                                                                 'Bad debt':'4'}).astype(int)
X_train_pb_['CREDIT_CURRENCY'] = X_train_pb_['CREDIT_CURRENCY'].map({'nan':'10', 'currency 1':'11',
                                                                 'currency 2':'12', 'currency 3':'13',
                                                                 'currency 4':'14'}).astype(int)
X_train_pb_['CREDIT_TYPE'] = X_train_pb_['CREDIT_TYPE'].map({'nan':'20', 'Consumer credit':'21',
       'Car loan':'22', 'Credit card':'23', 'Mortgage':'24',
       'Microloan':'25', 'Unknown type of loan':'26',
       'Loan for working capital replenishment':'27', 'Another type of loan':'28',
       'Loan for business development':'29',
       'Loan for the purchase of equipment':'30', 'Cash loan (non-earmarked)':'31',
       'Loan for purchase of shares (margin lending)':'32'}).astype(int)

In [ ]:
X_train_pb_.loc[X_train_pb_['CREDIT_CURRENCY'].isna(), 'CREDIT_CURRENCY'] = 'nan'
X_train_pb_['CREDIT_CURRENCY'].unique()

In [ ]:
X_train_pb_.loc[X_train_pb_['CREDIT_TYPE'].isna(), 'CREDIT_TYPE'] = 'nan'
X_train_pb_['CREDIT_TYPE'].unique()

In [ ]:
X_train_pb_.to_csv('X_train_pb_.csv')

In [ ]:
X_test_pb_.loc[X_test_pb_['CREDIT_ACTIVE'].isna(), 'CREDIT_ACTIVE'] = 'nan'
X_test_pb_.loc[X_test_pb_['CREDIT_CURRENCY'].isna(), 'CREDIT_CURRENCY'] = 'nan'
X_test_pb_.loc[X_test_pb_['CREDIT_TYPE'].isna(), 'CREDIT_TYPE'] = 'nan'

In [ ]:
X_test_pb_.to_csv('X_test_pb_.csv')

In [ ]:
X_trainn, X_testt, y_trainn, y_testt = train_test_split(X_train_pb_, y_train, shuffle=True, test_size=0.2, random_state=21)

In [ ]:
model_catb = catb.CatBoostClassifier(cat_features=['NAME_CONTRACT_TYPE', 'GENDER', 'EDUCATION_LEVEL',
                                                   'FAMILY_STATUS', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
                                                   'CREDIT_TYPE'], silent=True, random_state=21)
model_catb.fit(X_trainn, y_trainn)

y_train_pred = model_catb.predict(X_trainn)
y_test_pred = model_catb.predict(X_testt)

get_classification_report(y_trainn, y_train_pred, y_testt, y_test_pred)

In [ ]:
print(roc_auc_score(y_trainn, model_catb.predict_proba(X_trainn)[:,1]))
print(roc_auc_score(y_testt, model_catb.predict_proba(X_testt)[:,1]))

In [ ]:
TARGET = pd.DataFrame(model_catb.predict_proba(X_test_pb_))
TARGET =TARGET.rename(columns={0:0, 1:'TARGET'}).drop(0, axis=1)
TARGET.head()

In [ ]:
X_test = X_test.drop(['TARGET'], axis=1)

X_test['TARGET'] = TARGET

In [ ]:
X_test.head()

In [ ]:
X_test.to_csv('test_submission.csv', index=False)

In [ ]:
X_train_pb_['FLAG'] = 1
X_test_pb_['FLAG'] = 1

In [ ]:
X_train_pba = pd.merge(X_train_pb_, X_application_history, how='outer')
X_train_pba = X_train_pba.sort_values(by = ['APPLICATION_NUMBER'])
X_train_pba = X_train_pba.loc[~X_train_pba['FLAG'].isna()]
X_train_pba = X_train_pba.reset_index(drop=True)
X_train_pba.shape

In [ ]:
X_test_pba = pd.merge(X_test_pb_, X_application_history, how='outer')
X_test_pba = X_test_pba.sort_values(by = ['APPLICATION_NUMBER'])
X_test_pba = X_test_pba.loc[~X_test_pba['FLAG'].isna()]
X_test_pba = X_test_pba.reset_index(drop=True)
X_test_pba.shape

In [ ]:
X_train_pba_ = X_train_pba.drop_duplicates(subset=['APPLICATION_NUMBER'], keep='first')
X_train_pba_.shape

In [ ]:
X_test_pba_ = X_test_pba.drop_duplicates(subset=['APPLICATION_NUMBER'], keep='first')
X_test_pba_.shape

In [ ]:
X_train_pba_.loc[X_train_pba_['NAME_CONTRACT_STATUS'].isna(), 'NAME_CONTRACT_STATUS'] = 'nan'
X_train_pba_.loc[X_train_pba_['NAME_PAYMENT_TYPE'].isna(), 'NAME_PAYMENT_TYPE'] = 'nan'
X_train_pba_.loc[X_train_pba_['CODE_REJECT_REASON'].isna(), 'CODE_REJECT_REASON'] = 'nan'
X_train_pba_.loc[X_train_pba_['NAME_TYPE_SUITE'].isna(), 'NAME_TYPE_SUITE'] = 'nan'
X_train_pba_.loc[X_train_pba_['NAME_CLIENT_TYPE'].isna(), 'NAME_CLIENT_TYPE'] = 'nan'
X_train_pba_.loc[X_train_pba_['NAME_GOODS_CATEGORY'].isna(), 'NAME_GOODS_CATEGORY'] = 'nan'
X_train_pba_.loc[X_train_pba_['NAME_PORTFOLIO'].isna(), 'NAME_PORTFOLIO'] = 'nan'
X_train_pba_.loc[X_train_pba_['NAME_PRODUCT_TYPE'].isna(), 'NAME_PRODUCT_TYPE'] = 'nan'
X_train_pba_.loc[X_train_pba_['NAME_YIELD_GROUP'].isna(), 'NAME_YIELD_GROUP'] = 'nan'

In [ ]:
X_test_pba_.loc[X_test_pba_['NAME_CONTRACT_STATUS'].isna(), 'NAME_CONTRACT_STATUS'] = 'nan'
X_test_pba_.loc[X_test_pba_['NAME_PAYMENT_TYPE'].isna(), 'NAME_PAYMENT_TYPE'] = 'nan'
X_test_pba_.loc[X_test_pba_['CODE_REJECT_REASON'].isna(), 'CODE_REJECT_REASON'] = 'nan'
X_test_pba_.loc[X_test_pba_['NAME_TYPE_SUITE'].isna(), 'NAME_TYPE_SUITE'] = 'nan'
X_test_pba_.loc[X_test_pba_['NAME_CLIENT_TYPE'].isna(), 'NAME_CLIENT_TYPE'] = 'nan'
X_test_pba_.loc[X_test_pba_['NAME_GOODS_CATEGORY'].isna(), 'NAME_GOODS_CATEGORY'] = 'nan'
X_test_pba_.loc[X_test_pba_['NAME_PORTFOLIO'].isna(), 'NAME_PORTFOLIO'] = 'nan'
X_test_pba_.loc[X_test_pba_['NAME_PRODUCT_TYPE'].isna(), 'NAME_PRODUCT_TYPE'] = 'nan'
X_test_pba_.loc[X_test_pba_['NAME_YIELD_GROUP'].isna(), 'NAME_YIELD_GROUP'] = 'nan'

In [ ]:
y_train = X_train['TARGET']

In [ ]:
X_trainn, X_testt, y_trainn, y_testt = train_test_split(X_train_pba_, y_train, shuffle=True, test_size=0.2, random_state=21)

In [ ]:
model_catb = catb.CatBoostClassifier(cat_features=['NAME_CONTRACT_TYPE', 'GENDER', 'EDUCATION_LEVEL',
                                                   'FAMILY_STATUS', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
                                                   'CREDIT_TYPE',
                                                    'NAME_CONTRACT_STATUS',
                                                    'NAME_PAYMENT_TYPE',
                                                    'CODE_REJECT_REASON',
                                                     'NAME_TYPE_SUITE',
                                                     'NAME_CLIENT_TYPE',
                                                     'NAME_GOODS_CATEGORY',
                                                     'NAME_PORTFOLIO',
                                                     'NAME_PRODUCT_TYPE',
                                                     'NAME_YIELD_GROUP'], silent=True, random_state=21)
model_catb.fit(X_trainn, y_trainn)

y_train_pred = model_catb.predict(X_trainn)
y_test_pred = model_catb.predict(X_testt)

get_classification_report(y_trainn, y_train_pred, y_testt, y_test_pred)

In [ ]:
print(roc_auc_score(y_trainn, model_catb.predict_proba(X_trainn)[:,1]))
print(roc_auc_score(y_testt, model_catb.predict_proba(X_testt)[:,1]))

In [ ]:
TARGET = pd.DataFrame(model_catb.predict_proba(X_test_pba_))
TARGET =TARGET.rename(columns={0:0, 1:'TARGET'}).drop(0, axis=1)
TARGET.head()

In [ ]:
X_test = X_test.drop(['NAME_CONTRACT_TYPE'], axis=1)

X_test['TARGET'] = TARGET
X_test.head()

In [ ]:
X_test.to_csv('test_submission.csv', index=False)

In [ ]:
X_payments.dtypes

In [ ]:
X_train_pbap = pd.merge(X_train_pba_, X_payments, how='outer')
X_train_pay = X_train_pay.sort_values(by = ['APPLICATION_NUMBER'])
X_train_pay = X_train_pay.loc[~X_train_pay['FLAG'].isnull()]
X_train_pay = X_train_pay.reset_index(drop=True)
X_train_pay.shape

In [ ]:
X_test_pay = pd.merge(X_test, X_payments, how='outer')
X_test_pay = X_test_pay.sort_values(by = ['APPLICATION_NUMBER'])
X_test_pay = X_test_pay.loc[~X_test_pay['FLAG'].isnull()]
X_test_pay = X_test_pay.reset_index(drop=True)
X_test_pay.shape

In [ ]:
X_train_pay = X_train_pay.drop(['FLAG','TARGET'], axis=1)
X_test_pay = X_test_pay.drop(['FLAG'], axis=1)

In [ ]:
X_train_pay_ = X_train_pay.drop_duplicates(subset=['APPLICATION_NUMBER'], keep='first')
X_train_pay_.shape

In [ ]:
X_test_pay_ = X_test_pay.drop_duplicates(subset=['APPLICATION_NUMBER'], keep='first')
X_test_pay_.shape

In [ ]:
X_trainn, X_testt, y_trainn, y_testt = train_test_split(X_train_pay_, y_train, shuffle=True, test_size=0.2, random_state=21)

In [ ]:
model_catb = catb.CatBoostClassifier(cat_features=['NAME_CONTRACT_TYPE',
                                                    ], silent=True, random_state=21)
model_catb.fit(X_trainn, y_trainn)

y_train_pred = model_catb.predict(X_trainn)
y_test_pred = model_catb.predict(X_testt)

get_classification_report(y_trainn, y_train_pred, y_testt, y_test_pred)

## Обзор данных - EDA

In [ ]:
X_train_p['TARGET'] = pd.DataFrame(y_train)

Категориальные переменные

In [ ]:
categorical_features = X_train_p.select_dtypes(include=['object'])
print(f"count of categorical_features {categorical_features.shape[1]}")

cat_features = categorical_features.columns.to_list()
cat_features

###### Анализ числовых переменных

In [ ]:
numerical_features = X_train_p.select_dtypes(exclude=['object'])
print(f"count of numeric_features {numerical_features.shape[1]}")

numerical_features.columns

21 фичи из 26 - цифровые

###### Выявление дискретных признаков

In [ ]:
discrete_feature = [
    feature for feature in numerical_features
    if len(X_train_p[feature].unique())<25
]
print(f"Discrete Variables Count: {len(discrete_feature)}")

In [ ]:
X_train_p[discrete_feature].head()

10 фич из 21 - дискретные

Постараемся найти зависимость между дескретными признаками и целевой переменной

In [ ]:
fig, axes = plt.subplots(6, 2, figsize=(20, 30))

for num, feature in enumerate(discrete_feature):
    data = X_train_p.copy()
    statistics = data.groupby([feature])["TARGET"].max()

    sns.barplot(
        x=statistics.index,
        y=statistics.values,
        palette="viridis",
        ax=axes[num%6, num//6]
    )

Famili_size = 13 человек - нет дефолта
Childrens = 11 - нет деф
AMT_REQ_CREDIT_BUREAU_YEAR = 22 - нет деф

Default:
AMT_REQ_CREDIT_BUREAU_DAY: 0 - 2
Childrens: 0 - 6
AMT_REQ_CREDIT_BUREAU_WEEK: 0 - 3
AMT_REQ_CREDIT_BUREAU_MON: 0 - 11
AMT_REQ_CREDIT_BUREAU_QRT: 0 - 7
Famili_size: 1 - 8
AMT_REQ_CREDIT_BUREAU_YEAR: 0 - 12, 14
AMT_REQ_CREDIT_BUREAU_HOUR: 0 - 2

###### Непрерывные признаки

In [ ]:
continuous_feature = [
    feature for feature in numerical_features
    if feature not in discrete_feature
]

print(f"Continuous Feature Count {len(continuous_feature)}")

Для непрерывные признаков имеет смысл строить гистрограммы распределения или функции распределения, а также зависимость непрерывного признака в разрезе целевой переменной. Кроме того, имеет смысл строить распределение одновременно и для обучающей выборки, и для тестовой для понимания насколько отличаются распределения.

In [ ]:
# гистограммы распределения признаков:

fig, axes = plt.subplots(6, 2, figsize=(20, 30))

for num, feature in enumerate(continuous_feature):
    data = X_test_p[feature].copy()
    axes[num//2, num%2].hist(data, bins=50, color="blue", ec='white')
    axes[num//2, num%2].set_xlabel(feature, fontsize=10)

в тесте - что-то в зарплате:

In [ ]:
Выводы: "Own_car_age" - 64 и 65 - выбросы, лучше заменить на медиану
    "Day_on_last_job" - 365тыс дней - выброс, заменим на моду

In [ ]:
fig, axes = plt.subplots(6, 2, figsize=(20, 20))

for num, feature in enumerate(continuous_feature):
    try:
        train_data, test_data = X_train_p[feature].copy(), X_test_p[feature].copy()
        sns.kdeplot(train_data, ax=axes[num//2, num%2], color="blue", label="train")
        sns.kdeplot(test_data, ax=axes[num//2, num%2], color="green", label="test")
    except RuntimeError:
        pass
    except KeyError:
        train_data = train[feature].copy()
        sns.kdeplot(train_data, ax=axes[num//2, num%2], color="blue", label="train")
    axes[num//2, num%2].set_title(feature, fontsize=14)

вывод: train and test взяты из одного сета транзакции
    есть разница только в зарплатах

In [ ]:
correlation = numerical_features.corr()
corr_with_target = correlation["TARGET"].sort_values(ascending = False)
corr_with_target

In [ ]:
y_train = X_train[['TARGET']]

In [ ]:
final_iv, IV = data_vars(X_train_p, y_train) # вычисление уровней IV (Information Values) метода WOE по фичам

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2)
ax1, ax2 = ax
ax1.scatter(train["TransactionDT"], train["TransactionAmt"])
ax2.scatter(test["TransactionDT"], test["TransactionAmt"])
fig.set_size_inches(25, 5)

по результатам видим, что по величине транзакции по DT - разное распределение, на трене - 12 дней, на тесте 18, из-за разницы в количестве данных в трейне и тесте

In [ ]:
feature = "TransactionDT"

feature_with_target_s = train[[feature, 'isFraud']].sample(50000)
feature_s = feature_with_target_s[feature]
feature_target_0 = feature_s[feature_with_target_s['isFraud'] == 0]
feature_target_1 = feature_s[feature_with_target_s['isFraud'] == 1]
sns.kdeplot(feature_target_0, color="blue", label="target=0")
sns.kdeplot(feature_target_1, color="green", label="target=1")
feature_with_target_s_ = test[[feature, 'isFraud']].sample(75000)
feature_s_ = feature_with_target_s_[feature]
feature_target_0_ = feature_s_[feature_with_target_s_['isFraud'] == 0]
feature_target_1_ = feature_s_[feature_with_target_s_['isFraud'] == 1]
sns.kdeplot(feature_target_0_, color="red", label="target=0")
sns.kdeplot(feature_target_1_, color="yellow", label="target=1")

Выводы: для TransactionDT распределения в тесте и трейне крайне разные, что может
    привести к проблемам

In [ ]:
feature = "TransactionAmt"

feature_with_target_s = train[[feature, 'isFraud']].sample(50000)
feature_s = feature_with_target_s[feature]
feature_target_0 = feature_s[feature_with_target_s['isFraud'] == 0]
feature_target_1 = feature_s[feature_with_target_s['isFraud'] == 1]
sns.kdeplot(np.log(feature_target_0), color="blue", label="target=0")
sns.kdeplot(np.log(feature_target_1), color="green", label="target=1")
feature_with_target_s_ = test[[feature, 'isFraud']].sample(75000)
feature_s_ = feature_with_target_s_[feature]
feature_target_0_ = feature_s[feature_with_target_s_['isFraud'] == 0]
feature_target_1_ = feature_s[feature_with_target_s_['isFraud'] == 1]
sns.kdeplot(np.log(feature_target_0_), color="red", label="target=0")
sns.kdeplot(np.log(feature_target_1_), color="yellow", label="target=1")

для TransactionAmt мы видим более-менее совпадающие распределения в тесте и трейне,
разница в распределениях для 0 и 1 связано видимо с малым количеством данных по 1,,
скажем так - фрод хорошо маскируется по величине денег в транзакциях под норм (небольшие) транзакции

###### Категориальные признаки

In [ ]:
categorical_features = X_train.select_dtypes(include='object')
print(f"Categorical Feature Count {categorical_features.shape[1]}")
categorical_features.head()

In [ ]:
X_train['ProductCD'].unique()

In [ ]:
X_train['ProductCD'] = X_train['ProductCD'].map({'W':'0', 'H':'1', 'C':'2', 'S':'3', 'R':'4'}).astype(int)

In [ ]:
X_test['ProductCD'] = X_test['ProductCD'].map({'W':'0', 'H':'1', 'C':'2', 'S':'3', 'R':'4'}).astype(int)

In [ ]:
feature = "ProductCD"

feature_with_target_s = X_train[[feature, 'isFraud']].sample(50000)
feature_s = feature_with_target_s[feature]
feature_target_0 = feature_s[feature_with_target_s['isFraud'] == 0]
feature_target_1 = feature_s[feature_with_target_s['isFraud'] == 1]
sns.kdeplot(feature_target_0, color="blue", label="target=0")
sns.kdeplot(feature_target_1, color="green", label="target=1")
feature_with_target_s_ = X_test[[feature, 'isFraud']].sample(75000)
feature_s_ = feature_with_target_s_[feature]
feature_target_0_ = feature_s_[feature_with_target_s_['isFraud'] == 0]
feature_target_1_ = feature_s_[feature_with_target_s_['isFraud'] == 1]
sns.kdeplot(feature_target_0_, color="red", label="target=0")
sns.kdeplot(feature_target_1_, color="yellow", label="target=1")

распределение ProductCD разное для цели=0 для теста и трейна, цель=1 почти одинакова в трейне и тесте

X_train['card6'].unique()

In [ ]:
X_train['card6'] = X_train['card6'].map({'credit':'0', 'debit':'1', 'debit or credit':'2', 'charge card':'3', 'nan':'4'}).astype(float)

In [ ]:
X_test['card6'] = X_test['card6'].map({'credit':'0', 'debit':'1', 'debit or credit':'2', 'charge card':'3', 'nan':'4'}).astype(float)

In [ ]:
feature = "card6"

feature_with_target_s = X_train[[feature, 'isFraud']].sample(50000)
feature_s = feature_with_target_s[feature]
feature_target_0 = feature_s[feature_with_target_s['isFraud'] == 0]
feature_target_1 = feature_s[feature_with_target_s['isFraud'] == 1]
sns.kdeplot(feature_target_0, color="blue", label="target=0")
sns.kdeplot(feature_target_1, color="green", label="target=1")
feature_with_target_s_ = X_test[[feature, 'isFraud']].sample(75000)
feature_s_ = feature_with_target_s_[feature]
feature_target_0_ = feature_s_[feature_with_target_s_['isFraud'] == 0]
feature_target_1_ = feature_s_[feature_with_target_s_['isFraud'] == 1]
sns.kdeplot(feature_target_0_, color="red", label="target=0")
sns.kdeplot(feature_target_1_, color="yellow", label="target=1")

распределения более менее совпадают

X_train['card4'].unique()

In [ ]:
X_train['card4'] = X_train['card4'].map({'discover':'0', 'mastercard':'1', 'visa':'2', 'american express':'3', 'nan':'4'}).astype(float)

In [ ]:
X_test['card4'] = X_test['card4'].map({'discover':'0', 'mastercard':'1', 'visa':'2', 'american express':'3', 'nan':'4'}).astype(float)

In [ ]:
feature = "card4"

feature_with_target_s = X_train[[feature, 'isFraud']].sample(50000)
feature_s = feature_with_target_s[feature]
feature_target_0 = feature_s[feature_with_target_s['isFraud'] == 0]
feature_target_1 = feature_s[feature_with_target_s['isFraud'] == 1]
sns.kdeplot(feature_target_0, color="blue", label="target=0")
sns.kdeplot(feature_target_1, color="green", label="target=1")
feature_with_target_s_ = X_test[[feature, 'isFraud']].sample(75000)
feature_s_ = feature_with_target_s_[feature]
feature_target_0_ = feature_s_[feature_with_target_s_['isFraud'] == 0]
feature_target_1_ = feature_s_[feature_with_target_s_['isFraud'] == 1]
sns.kdeplot(feature_target_0_, color="red", label="target=0")
sns.kdeplot(feature_target_1_, color="yellow", label="target=1")

распределения более менее совпадают

## Анализ пропусков

In [ ]:
msno.matrix(X_train, figsize=(20, 5))

очень много пропусков

In [ ]:
features_with_na = [
    feature for feature in X_train.columns
    if X_train[feature].isnull().sum() > 1]

fig, axes = plt.subplots(100, 4, figsize=(30, 400))
for num, feature in enumerate(features_with_na):
    data = train.copy()
    data[feature] = np.where(data[feature].isnull(), 1, 0)
    statistics = data.groupby(feature)["isFraud"].median()
    axes[num%100, num//100].bar(statistics.index, statistics.values)
    axes[num%100, num//100].set_xticks([0, 1])
    axes[num%100, num//100].set_xticklabels(["no missing", "missing"])
    axes[num%100, num//100].set_ylabel(feature)

In [ ]:
Итого: у следующих признаков имеются различия в распределениях в зависимости от присутствия и отсутствия значений:
        card3 и card6,можно сгенерировать новые признаки на этой основе

In [ ]:
missing_values = categorical_features.isnull().sum(axis=0).reset_index()
missing_values.columns = ['column_name', 'missing_count']
missing_values = missing_values.loc[missing_values['missing_count']>25000]
missing_values = missing_values.sort_values(by='missing_count')

ind = np.arange(missing_values.shape[0])
fig, axes = plt.subplots(figsize=(8, 4))
rects = axes.barh(ind, missing_values.missing_count.values, height=0.9)
axes.set_yticks(ind)
axes.set_yticklabels(missing_values.column_name.values, rotation='horizontal')
axes.set_title("Missing Observations Count - Categorical Features")
axes.set_xlabel("Missing Observations Count")

Если отбросить по принципу 50%, то нужно отбросить: M4, M1, M2, M3, M5, R_emaildomain, M7, M8, M9

In [ ]:
missing_values = numerical_features.isnull().sum(axis=0).reset_index()
missing_values.columns = ['column_name', 'missing_count']
missing_values = missing_values.loc[missing_values['missing_count']<50000]
missing_values = missing_values.sort_values(by='missing_count')

ind = np.arange(missing_values.shape[0])
fig, axes = plt.subplots(figsize=(8, 80))
rects = axes.barh(ind, missing_values.missing_count.values, height=0.9)
axes.set_yticks(ind)
axes.set_yticklabels(missing_values.column_name.values, rotation='horizontal')
axes.set_title("Missing Observations Count - Categorical Features")
axes.set_xlabel("Missing Observations Count")

In [ ]:
Если отбросить по принципу 50%, то нужно отбросить 183 фичи, практически половину

###### Анализ "скошенности" распределений:

In [ ]:
fig = plt.figure(figsize=(150, 5))
train_stats, test_stats = X_train.skew(), X_test.skew()
plt.plot(train_stats, color="blue", label="train")
plt.plot(test_stats, color="green", label="test")
plt.xticks(range(train_stats.shape[0]), train_stats.index, rotation=90)
plt.xlabel("features", size=15)
plt.xlim(0, len(train_stats))
plt.ylabel("skew", size=15)
plt.show()

In [ ]:
Кандидаты на удаление: adr2, c3, c7, c12, v1, v14, v27,28,68,89,117,118,119,126,128,135,136,137,202,204,214,215,216,221,222,
    226,240,241,263,276,277,278,305,306,319,320,321

In [ ]:
fig = plt.figure(figsize=(150, 5))
train_stats, test_stats = X_train.kurtosis(), X_test.kurtosis()
plt.plot(train_stats, color="blue", label="train")
plt.plot(test_stats, color="green", label="test")
plt.xticks(range(train_stats.shape[0]), train_stats.index, rotation=90)
plt.xlabel("features", size=15)
plt.ylabel("kurtosis", size=15)
plt.xlim(0, len(train_stats))
plt.show()

Кандидаты на удаление: adr2, c3, c7, c12, v1, v27,28,68,89,117,118,119,126,128,135,136,137,202,204,214,215,216,221,222,
    226,240,241,263,276,277,278,305,306,319,320,321

## Анализ распределения целевой переменной

###### Проверка баланса классов цели¶

In [ ]:
X_train['isFraud'].value_counts()

In [ ]:
plt.figure(figsize=(4, 2))

sns.countplot(x='isFraud', data=X_train)

plt.title('Target variable distribution')
plt.show()

Вывод: крайне сильный дисбаланс, надо делать балансировку

### Для дальнейшего изучения взаимосвязи признаков будем использовать:
* Correlation Heat Map
* Heat Map для отдельных признаков
* Pair Plot
* Scatter Plot

## Корреляция между числовыми признаками

построим график зависимости количества коррелированных признаков от уровня кореляции (признаков с уровнем кореляции выше чем уровень корреляции на этом шаге) 

In [ ]:
correlation = numerical_features.corr()
x = []
y = []
i = 0.01
while i < 0.2:
    x.append(len(correlation.loc[(correlation["isFraud"] > i)|(correlation["isFraud"] < -i), "isFraud"]))
    y.append(i)
    i += 0.01
plt.scatter(y,x)
plt.show()

Вывод: выбор фич по уровню 0.2 в данном случае не подходит, тк. имеет место зависимость, близкая к линейной - не очень хорошая идея отбрасывать много коррелированных фич

In [ ]:
# Тепловая карта - получаем информацию о силе связи между переменными.
fig, axes = plt.subplots(figsize = (50,50))
plt.title("Correlation of Numeric Features with isFraud", y=1, size=16)
sns.heatmap(correlation, square=True, vmax=0.8, cmap="viridis")

In [ ]:
Многие фичи очень сильно коррелированы друг с другом, можно подумать об удалении части из них

In [ ]:
msno.heatmap(X_train, figsize=(200, 120))

Присутствует мощнейшая кореляция, после анализа, выводы - следующие диапазоны можно
заменить одним признаком:
    - M1,M2,dist1
    - D9,D8
    - adr1,adr2
    - card2,card6
    - M7,M8
    - V1-V10
    - V12-V33
    - V35-V51
    - V53-V73
    - V75-V93
    - V138-V165
    - V167-V277
    - V279-V320
    - V322-V339

###### теперь посмотрим в сравнении трейн и тест:

In [ ]:
train_correlations = numerical_features.corr()
train_correlations = train_correlations.values.flatten()
train_correlations = train_correlations[train_correlations != 1]

columns = numerical_features.columns.tolist()
columns.remove("isFraud")

test_correlations = X_test[columns].corr()
test_correlations = test_correlations.values.flatten()
test_correlations = test_correlations[test_correlations != 1]

plt.figure(figsize=(20,5))
sns.distplot(train_correlations, color="Blue", label="train")
sns.distplot(test_correlations, color="Green", label="test")

plt.xlabel("Correlation values found in train (except 1)", size=14)
plt.title("Are there correlations between features?", size=14)
plt.legend(loc="best", fontsize=14)
plt.ylabel("Density", size=14)

трейн и тест более менее одинаково показывают линейную корреляцию фичей к цели -> трейн и тест из одного сета,
модель должна быть стабильна в логистической регрессии

## Pair Plot

PairPlor между `target` и высоко-коррелированными переменными. Строим этот график для лучшего понимания характерика связи между целевой переменной и признаком. С помощью корреляционной матрицы мы установили что связь между определенными признаками и целевой переменной есть, но характер этой связи пока остается невыясненным. Строим PairPlot для лучшего понимания характера этой связи;

In [ ]:
sns.set()
columns = [
    "isFraud", "V201", "V189", "V200", "V198", "V156",
    "V257", "V158", "V188", "V155", "V45", "V149", "V231", "V197",
    "V157", "V199", "V233", "V147", "V148", "V171", "V52", "V87", "V146", "TransactionDT"
]

sns.pairplot(X_train[columns], size=2, kind="scatter", diag_kind="kde")
plt.show()


__Короткий анализ:__

Существует много линейных зависимости между фиами
Существует много линейных границ между фичами, выше или ниже которых другая фича не идёт


###### Теперь оценим фичи по методу WOE/IV (выбираем в диапазоне от 0.1 до 0.5)


In [ ]:
final_iv, IV = data_vars(X_train, X_train['isFraud']) # вычисление уровней IV (Information Values) метода WOE по фичам

In [ ]:
IV.loc[(IV['IV'] > 0.1), :].sort_values('IV')

Итак, у нас 184 значимых фичи из 379. При этом, M4, C12, R_emaildomain - слишком хороши, чтобы остаться в пуле, тем более ранее мы их определили кандидатами на выбывание, остаётся 181 фича, по графику корреляции это соответствует уровню 0.065, сравним эти 2 списка

In [ ]:
df_correlation=pd.DataFrame(correlation.loc[(correlation["isFraud"] > 0.065)|(correlation["isFraud"] < -0.065), "isFraud"])

In [ ]:
df_correlation.to_csv('df_correlation.csv', sep=';')
IV.loc[(IV['IV'] > 0.1), :].sort_values('IV').to_csv('IV.csv', sep=';', index=False)

После обработки в Excel (слияние, удаление дубликатов, удаление излишеих фич после анализа корреляции):

In [ ]:
df_merge_cut = pd.read_csv('merge_cut.csv', sep=';').to_numpy().flatten()
df_merge_cut

In [ ]:
X = train['isFraud']
for i in range(0,62):
    X = pd.concat([X, train[df_merge_cut[i]]], axis=1)
X.head()

In [ ]:
X = test['isFraud']
for i in range(0,62):
    X = pd.concat([X, train[df_merge_cut[i]]], axis=1)
X.head()

In [ ]:
X_tr = train[df_merge_cut[0]]
for i in range(1,62):
    X_tr = pd.concat([X_tr, train[df_merge_cut[i]]], axis=1)

X_tr = X_tr.select_dtypes(include=[np.number])
X_tr.head()

In [ ]:
X_te = test[df_merge_cut[0]]
for i in range(1,62):
    X_te = pd.concat([X_te, test[df_merge_cut[i]]], axis=1)
    
X_te = X_te.select_dtypes(include=[np.number])
X_te.head()

In [ ]:
msno.heatmap(X, figsize=(20, 12))

###### Теперь проверим распределения выбранных фич в зависимости от целевой функции "0" или "1" 

In [ ]:
features = X.select_dtypes(include=[np.number])

fig, axes = plt.subplots(20, 3, figsize=(15, 60))
for num, feature in enumerate(features):
    try:
        feature_with_target_s = X[[feature, 'isFraud']].sample(1000)
        feature_s = feature_with_target_s[feature]
        feature_target_0 = feature_s[feature_with_target_s['isFraud'] == 0]
        feature_target_1 = feature_s[feature_with_target_s['isFraud'] == 1]
        sns.kdeplot(feature_target_0, ax=axes[num//3, num%3], color="blue", label="target=0")
        sns.kdeplot(feature_target_1, ax=axes[num//3, num%3], color="green", label="target=1")
    except RuntimeError:
        pass
    except KeyError:
        train_data = X_train[feature].copy()
        sns.kdeplot(feature_target_0, ax=axes[num//3, num%3], color="blue", label="target=0")
        sns.kdeplot(feature_target_1, ax=axes[num//3, num%3], color="green", label="target=1")
    axes[num//3, num%3].set_title(feature, fontsize=10)   

Вывод: у всех фич наблюдаются различия в распределениях, отсутствующие графики свидетельствуют об остуствии данных по 0 или 1 в цели или об очень маленьких величинах

In [ ]:
# Теперь посмотрим визуально на распределение в трейне и тесте, есть ли различия

fig, axes = plt.subplots(80, 2, figsize=(15, 300))
columns = X.select_dtypes(include=[np.number])

for num, feature in enumerate(columns):
    try:
        train_data, test_data = train[feature].copy(), test[feature].copy()
        sns.kdeplot(train_data, ax=axes[num//2, num%2], color="blue", label="train")
        sns.kdeplot(test_data, ax=axes[num//2, num%2], color="green", label="test")
    except RuntimeError:
        pass
    except KeyError:
        train_data = train[feature].copy()
        sns.kdeplot(train_data, ax=axes[num//2, num%2], color="blue", label="train")
    axes[num//2, num%2].set_title(feature, fontsize=10)

## Анализ распределения на обучении и тесте

Совпадение распределений на выборке для обучения и для тестирования - одна из ключевых коцепций статистического обучения. Обучая модель на некоторых данных, мы принимаем гипотезу о том, что модель должна работать на таких же данных.

## Adversarial Validation

In [ ]:
X_tra = X_train.select_dtypes(include=[np.number])

In [ ]:
X_tes = X_test.select_dtypes(include=[np.number])

In [ ]:
X_tra = X_tra.drop(['TransactionDT','TransactionID'], axis=1)
X_tes = X_tes.drop(['TransactionDT','TransactionID'],  axis=1)

In [ ]:
X_adv = pd.concat([X_tra, X_tes])

y_adv = np.hstack(
    (np.ones(X_train.shape[0]), np.zeros(X_test.shape[0])))

X_adv = X_adv.reset_index(drop=True)

In [ ]:
%%time
parameters = {"max_depth": 5, "n_estimators": 50, "random_state": 27, "n_jobs": 2}
model = xgb.XGBClassifier(**parameters)

cv = cross_val_score(
    estimator=model,
    X=X_adv, y=y_adv,
    scoring="roc_auc",
    cv=KFold(n_splits=3, shuffle=True, random_state=27)
)

print(f"CV-score: {round(np.mean(cv), 4)}")

In [ ]:
X_adv = pd.concat([X_tr, X_te])

y_adv = np.hstack(
    (np.ones(X_train.shape[0]), np.zeros(X_test.shape[0])))

X_adv = X_adv.reset_index(drop=True)

In [ ]:
%%time
parameters = {"max_depth": 5, "n_estimators": 50, "random_state": 27, "n_jobs": 2}
model = xgb.XGBClassifier(**parameters)

cv = cross_val_score(
    estimator=model,
    X=X_adv, y=y_adv,
    scoring="roc_auc",
    cv=KFold(n_splits=3, shuffle=True, random_state=27)
)

print(f"CV-score: {round(np.mean(cv), 4)}")

In [ ]:
Окончательно убедились, что трейн и тест из одного датасета и практически статистически неразличимы

In [ ]:
import gc
gc.collect()

In [ ]:
model.fit(X_adv, y_adv)

importance = pd.DataFrame({
    "feature": X_adv.columns,
    "adversarial_importance": model.feature_importances_
})
importance = importance.sort_values(by="adversarial_importance", ascending=False)
importance = importance.head(n=30)

plt.figure(figsize=(20, 5))
sns.barplot(x=importance["feature"], y=importance["adversarial_importance"], palette="viridis")
plt.title("Top Importance Features for Adversarial Validation (train vs test)", size=14)

Валидация